# Analysis of Traffic Data in Chicago from 2013-2020


# Introduction

## Authors: 
## Kevin Thompson, Dilruksha Jayaweera, Heber Nielsen, Sangrae Cho, Kumaraiah Pradeepkumar

Car accidents are no trivial matter in the United States. Roughly 800 car accidents occur every day in Illinois alone [@tribune]. Our project is to develop models to both predict and explain a subset of these entirely avoidable accidents in Chicago, Illinois. OpenWeather's historical weather database and the city government of Chicago's website to construct these models and insights. We also obtained the boundaries of the police beats from the city of Chicago's website. These datasets include factors that are widely believed to contribute to higher accident rates, such as weather, time, location, road conditions, and so forth.

  The primary goal of our exploration is to explore useful features for predicting the number of accidents in a given police shift in location clusters called "police beats". We hope that the results of our project could be used to further optimize the allocation of first responders across these beats. We believe that the effectiveness of a good prediction algorithm would be measured using one of the many of the many scoring rules described in [@czado]. As of this moment,we find the Dawid-Sebastiani and squared error scores to be the most appealing. 
  
  We now turn to the specifics of the data.
 
# Data
  
  

In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
crash_data = pd.read_csv()


Note: We found that the crash data prior to 2017 was infrequently captured, which leads us to not trust it. Therefore, we decided to remove data prior to 2017. This is not really a problem because the vast majority of the data comes after 2017. We will also merge the data sets, which requires us to resample our time series component in the process because the weather data was captured hourly. If we allow for periods of infrequent sampling to sneak in, then it will make us think (possibly incorrectly) that our target is not covariance stationary. 

In [2]:
final_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 377630 entries, 0 to 377629
Data columns (total 48 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   RD_NO                          377630 non-null  object        
 1   CRASH_DATE_EST_I               28062 non-null   category      
 2   CRASH_DATE                     377630 non-null  datetime64[ns]
 3   POSTED_SPEED_LIMIT             377630 non-null  int64         
 4   TRAFFIC_CONTROL_DEVICE         377630 non-null  category      
 5   DEVICE_CONDITION               377630 non-null  category      
 6   WEATHER_CONDITION              377630 non-null  category      
 7   LIGHTING_CONDITION             377630 non-null  category      
 8   FIRST_CRASH_TYPE               377630 non-null  category      
 9   TRAFFICWAY_TYPE                377630 non-null  category      
 10  LANE_CNT                       198549 non-null  float64       
 11  

In [3]:
# percent missingness by variable
pd.set_option('precision', 2)
crashes_missing = crashes.isna()
crashes_missing_sum = crashes_missing.sum()
crashes_missing_pct = (crashes_missing_sum / len(crashes))*100
crashes_missing_pct


RD_NO                            0.00e+00
CRASH_DATE_EST_I                 9.26e+01
CRASH_DATE                       0.00e+00
POSTED_SPEED_LIMIT               0.00e+00
TRAFFIC_CONTROL_DEVICE           0.00e+00
DEVICE_CONDITION                 0.00e+00
WEATHER_CONDITION                0.00e+00
LIGHTING_CONDITION               0.00e+00
FIRST_CRASH_TYPE                 0.00e+00
TRAFFICWAY_TYPE                  0.00e+00
LANE_CNT                         4.74e+01
ALIGNMENT                        0.00e+00
ROADWAY_SURFACE_COND             0.00e+00
ROAD_DEFECT                      0.00e+00
REPORT_TYPE                      2.32e+00
CRASH_TYPE                       0.00e+00
INTERSECTION_RELATED_I           7.79e+01
NOT_RIGHT_OF_WAY_I               9.54e+01
HIT_AND_RUN_I                    7.22e+01
DAMAGE                           0.00e+00
DATE_POLICE_NOTIFIED             0.00e+00
PRIM_CONTRIBUTORY_CAUSE          0.00e+00
SEC_CONTRIBUTORY_CAUSE           0.00e+00
STREET_NO                        0

In [4]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63859 entries, 0 to 63858
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   dt_iso               63859 non-null  datetime64[ns]
 1   weather_main         63859 non-null  category      
 2   weather_description  63859 non-null  category      
dtypes: category(2), datetime64[ns](1)
memory usage: 625.6 KB


There are several variables that have a significant number of missing values, though we don't find most of these particularly interesting for our use cases. Doorings, accident, worker zone variables, and so forth may be useful outcome variables but they are not so interesting for our current investigation. We will thus remove these variables, other variables that we felt would lead to target leakage, and variables we feel are downright unrelated to predicting number of accidents in a given beat during a given shift. The weather variables are complete and contain no missing values. 

The features and outcomes we are interested in tend to have very few missing values.

The attributes in this resulting dataframe set can be divided into injury variables, weather variables, time variables, and the number of accidents in a given hour at a given beat. CRASH_DATE gives a datetime for the crash, all of the injury variables yield the number of people who are injured in that specific manner as a result of the accidents, HIT_AND_RUN_I tells us the number of hit and runs during that hour in that beat, and CRASH_HOUR is an integer variable from 0 to 24, with 0 = midnight. NUM_ACCIDENTS, in its current form gives us the number of accidents in an hour in a specific beat. dt_iso is the datetime of a weather incident in the ISO format. weather_main and weather_description describe the state of the weather at that specific time. For example, they might say "light snow" or "heavy rain". There is significant intuitive justification for believing that these variables significantly contribute to accident rates. 

# Summary

In [ ]:
crash_per_hour.describe()

Above we see summary statistics for the numeric variables in our dataset. There are somew extreme outliers in our dataset, but we could not find any evidence to indicate that they are mistakes. Furthermore, is entirely plausible that there could be accidents that involve 209 people in a city as dense and populous as Chicago. It is furthermore clear that all of our variables have a significant right skew, which is further confirmation to us that the outliers merely reflect the underlying nature of the data generating process.

In [ ]:
categorical_vars = crashes_new.select_dtypes(include=['category'])
categorical_vars['weather_description'].value_counts()

In [ ]:
categorical_vars['weather_main'].value_counts()

Both of these variables follow standard intuition. The weather tends to be cloudy or clear with a significant number of snowy days due to Chicago's location and proximity to a large body of water.

# Important Attributes

At first glance, we believe that the injury and number of units variables are the most important. 

In [ ]:
plt.subplot(2,3,1)
plt.hist(crashes.INJURIES_TOTAL)
plt.title('Total Injuries')
plt.subplot(2,3,2)
plt.hist(crashes.INJURIES_FATAL)
plt.title("Fatal Injuries")
plt.subplot(2,3,3)
plt.hist(crashes.INJURIES_INCAPACITATING)
plt.title("Incapacitating Injuries")
plt.subplot(2,3,4)
plt.hist(crashes.INJURIES_NON_INCAPACITATING)
plt.title("Non-Incapacitating Injuries")
plt.subplot(2,3,5)
plt.hist(crashes.NUM_UNITS)
plt.title("Number of Units")

plt.tight_layout()

Histograms are great for count data, so we felt that this was the best way to visualizes injuries and number of units. Given the low counts, these poisson-esque distributions are to be expected; however, they may yet become more informative when we do some feature engineering and look at them from the perspective of rates. The number of units (people, automobiles, etc.) tends to hover around 1-4, which makes a great deal of sense. Given that fact, it makes sense that many of the injury variables tend to hover around 1-2.

# Interesting Relationships

We will group these variables by beat later on when we are interested in clustering our observations. For now, our data has time-varying components, space-varying components, and components that vary over both. There are several theoretical reasons to suppose that accident rates vary across time as well as location, so we have to first see if there is a stable time structure for our target variable. This will best be seen if we marginalize across the days and the beats.

In [ ]:
# Marginalize across days of the month to filter out noise for plotting

aggregations = dict.fromkeys(['NUM_UNITS', 'INJURIES_TOTAL', 'INJURIES_FATAL', 'INJURIES_INCAPACITATING',
                        'INJURIES_NON_INCAPACITATING', 'INJURIES_REPORTED_NOT_EVIDENT', 'INJURIES_NO_INDICATION',
                        'INJURIES_UNKNOWN', 'NUM_ACCIDENTS'], np.sum)
aggregations.update(dict.fromkeys(['weather_main', 'weather_description', 'HIT_AND_RUN_I'], 'size'))
crashes_over_time = crashes_new_daily.groupby(pd.Grouper(key='CRASH_DATE', freq='M')).agg(aggs2)


In [ ]:
# plot time series of monthly crashes
plt.plot(crashes_over_time.index, crashes_over_time.NUM_ACCIDENTS)

In this plot we can see that the time series plot is not covariance stationary across the time-horizon. There was a sudden structural break in the mean number of accidents per month starting in mid 2017. We currently do not have an explanation for this, but we believe that following up on this peculiarity will lead to better model performance in the future. It could be the case that this reflects even better data collection methods being adopted during 2017. If so, we could consider dropping 2017 as well to get a better view of the series's stationarity. 

Next we will see how accidents vary across beats.

In [ ]:
top_beats = crashes_new_daily.groupby('BEAT_OF_OCCURRENCE').agg({'NUM_ACCIDENTS':np.mean}).sort_values('NUM_ACCIDENTS',
                                                                                                      ascending=False)
#merge beats data
beats_combined = beats.merge(top_beats, how='left', left_on='beat_num', right_on='BEAT_OF_OCCURRENCE')
beats_combined.plot(column='NUM_ACCIDENTS', legend=True, figsize=(20,10))

Interestingly, we can see a very clear pattern here. South Chicago is safer to drive in, Northern Chicago is moderately risky to drive in, and there are a couple points in mid-eastern chicago that are far and above more dangerous than any other part of the city. This plot will probably be among the most helpful when it comes to model-building because we can see this clear spatial pattern that allows us cluster beats themselves. 

Now let's see how crashes behave across shifts.

In [ ]:
plt.figsize = (20,60)
plt.subplot(1,3,1)
plt.hist(crashes_new_daily.loc[crashes_new_daily.SHIFT==1].NUM_ACCIDENTS)
plt.title('SHIFT 1')
plt.subplot(1,3,2)
plt.hist(crashes_new_daily.loc[crashes_new_daily.SHIFT==2].NUM_ACCIDENTS)
plt.title('SHIFT 2')
plt.subplot(1,3,3)
plt.hist(crashes_new_daily.loc[crashes_new_daily.SHIFT==3].NUM_ACCIDENTS)
plt.title('SHIFT 3')
plt.tight_layout()


In [ ]:
crashes_new_daily.groupby('SHIFT').agg({'NUM_ACCIDENTS': np.mean})

Conditional on shift, the number of accidents appears to have a very clear poisson distribution. This is in accordance with expectation. The sheer niceness of this distributional form is convenient for modelling and the differences in mean number of accidents per shift is in accordance with intuition. It is far safer to drive at night and the riskiest time to be driving is when everyone is coming home from work. 

In [ ]:
pd.crosstab(crash_per_hour.WEATHER_MAIN, crash_per_hour.WEATHER_DESCRIPTION)

Here we see that light weather is far more common than heavy weather for all types of main_weather. This is interesting because it may be the case that because weather tends to mostly be light, the WEATHER_MAIN column may not explain as much as the WEATHER_DESCRIPTION column. After all, it's not that hard to drive in light weather. Adding more factors slows down algorithms, but it may be worth it here.

Next we will examine injuries to see if we can find some interesting relationships there.

In [ ]:
fig, ax = plt.subplots(1, 5, figsize = (25, 5));

sns.barplot(x="CRASH_MONTH",y="INJURIES_FATAL",  data=crashes, ax=ax[0]);
sns.barplot(x="CRASH_MONTH", y="INJURIES_INCAPACITATING",  data=crashes, ax=ax[1]);
sns.barplot(x="CRASH_MONTH", y="INJURIES_NON_INCAPACITATING",  data=crashes, ax=ax[2]);
sns.barplot(x="CRASH_MONTH", y="INJURIES_REPORTED_NOT_EVIDENT",  data=crashes, ax=ax[3]);
sns.barplot(x="CRASH_MONTH", y="INJURIES_NO_INDICATION",  data=crashes, ax=ax[4]);
[ax[i].set_xlabel('CRASH MONTH') for i in range(5)]
ax[0].set_title('Fatal Crash by month')
ax[1].set_title('Incapacitating Crash by month')
ax[2].set_title('Non-Incapacitating Crash by month')
ax[3].set_title('Reported not evident crash by month')
ax[4].set_title('No Indication Crash by month')

plt.show()
plt.tight_layout()

The degree of seasonality in fatalities is the most immediate thing you'll notice from these charts. The summer months are the most dangerous months of the year to be driving. This is counter to expectation, where we expected the winter months to be the most dangerous. After some reflection, a possible explanation would be that people are more careful when they are driving in the snow and are thus less likely to engage in behavior that endangers their life. They may also want to drive less because driving in the snow can be a hassle. 

In [ ]:
plt.figure(figsize=(15,6))
sns.barplot(y="WEATHER_CONDITION", x="INJURIES_FATAL",  data=crashes);
plt.tight_layout()



It makes intuitive sense that fog/smoke/haze would be riskier than clear weather and the data agrees with this intuition. What is interesting here is that snow is less risky than clear weather, yet rainy weather is more risky than clear weather. A possible explanation is that people overestimate the risks of driving in the snow and underestimate the risks of driving in the rain.

# Possible Feature Engineering

We believe that features that could be created from the data in the future include different rates of injury per number of units involved in a car accident. Danger is really about what it does to the people involved and it may be the case that something that is observed to be riskier is merely so because more people just so happened to be involved in the accidents at those times.